# World University Rankings 2023 #

## STAT 301 Group Project 

### Introduction

Start with relevant background information on the topic to prepare those unfamiliar for the rest of your proposal.

Formulate one or two questions for investigation and detail the dataset that will be utilized to address these questions.

Additionally, align your question/objectives with the existing literature. To contextualize your study, include a minimum of two scientific publications (these should be listed in the References section).



### Methods and Results

In this section, you will include:

**a) “Exploratory Data Analysis (EDA)”**

- Demonstrate that the dataset can be read into R.
- Clean and wrangle your data into a tidy format.
- Plot the relevant raw data, tailoring your plot to address your question.
  - Make sure to explore the association of the explanatory variables with the response.
- Any summary tables that are relevant to your analysis.
- Be sure not to print output that takes up a lot of screen space.
- Your EDA must be comprehensive with high quality plots.

**b) “Methods: Plan”**

- Describe in written English the methods you used to perform your analysis from beginning to end, and narrate the code that does the analysis.
- If included, describe the “Feature Selection” process and how and why you choose the covariates of your final model.
- Make sure to interpret/explain the results you obtain. It’s not enough to just say, “I fitted a linear model with these covariates, and my R-square is 0.87”.
  - If inference is the aim of your project, a detailed interpretation of your fitted model is required, as well as a discussion of relevant quantities (e.g., are the coefficients significant? How does the model fit the data)?
  - A careful model assessment must be conducted.
  - If prediction is the project's aim, describe the test data used or how it was created.
- Ensure your tables and/or figures are labelled with a figure/table number.

In [6]:
library(tidyverse)
library(repr)
library(broom)
library(GGally)
library(car)
library(rsample)
library(leaps)
library(glmnet)

In [95]:
# part a
#loading and fixing column names
university_data <- read_csv("uni_rankings_2023.csv")
colnames(university_data) <- c("university_rank", "name_of_university", "location", "no_of_student", "no_of_student_per_staff", 
                               "international_student", "female_male_ratio", "overall_score", "teaching_score", "research_score",
                               "citations_score","industry_income_score", "international_outlook_score")
head(university_data)

# changing columns with strings into numerical data

university_data_cleaned <- university_data |>
mutate(international_student = as.numeric(gsub("%", "", international_student)) / 100,
      female_male_ratio = as.numeric(sub(":.*", "", female_male_ratio))/as.numeric(sub(".*:", "", female_male_ratio)),
      overall_score = ifelse(grepl("–", overall_score),
                      rowMeans(apply(do.call(rbind, strsplit(gsub("–", "-", overall_score), "-")), 2, as.numeric), na.rm = TRUE),
                      as.numeric(overall_score)), #coded with help from hyperskill.org, stackoverflow, chatgpt
      teaching_score = as.numeric(teaching_score),
      research_score = as.numeric(research_score),
      citations_score = as.numeric(citations_score),
      industry_income_score = as.numeric(industry_income_score),
      international_outlook_score = as.numeric(international_outlook_score)) |>
drop_na()

#filtering all rows to removw Inf and NaN
university_data_cleaned <- university_data_cleaned %>%
  filter_all(all_vars(!is.infinite(.) & !is.nan(.)))

#number of rows
nrow(university_data_cleaned)

head(university_data_cleaned)

Rows: 2341 Columns: 13
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (11): University Rank, Name of University, Location, International Stude...
dbl  (1): No of student per staff
num  (1): No of student

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


university_rank,name_of_university,location,no_of_student,no_of_student_per_staff,international_student,female_male_ratio,overall_score,teaching_score,research_score,citations_score,industry_income_score,international_outlook_score
<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,University of Oxford,United Kingdom,20965,10.6,42%,48 : 52,96.4,92.3,99.7,99.0,74.9,96.2
2,Harvard University,United States,21887,9.6,25%,50 : 50,95.2,94.8,99.0,99.3,49.5,80.5
3,University of Cambridge,United Kingdom,20185,11.3,39%,47 : 53,94.8,90.9,99.5,97.0,54.2,95.8
3,Stanford University,United States,16164,7.1,24%,46 : 54,94.8,94.2,96.7,99.8,65.0,79.8
5,Massachusetts Institute of Technology,United States,11415,8.2,33%,40 : 60,94.2,90.7,93.6,99.8,90.9,89.3
6,California Institute of Technology,United States,2237,6.2,34%,37 : 63,94.1,90.9,97.0,97.3,89.8,83.6


Warning message:
“There were 8 warnings in `mutate()`.
The first warning was:
ℹ In argument: `overall_score = ifelse(...)`.
Caused by warning in `apply()`:
! NAs introduced by coercion
ℹ Run `dplyr::last_dplyr_warnings()` to see the 7 remaining warnings.”


[1] 1481

university_rank,name_of_university,location,no_of_student,no_of_student_per_staff,international_student,female_male_ratio,overall_score,teaching_score,research_score,citations_score,industry_income_score,international_outlook_score
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,University of Oxford,United Kingdom,20965,10.6,0.42,0.9230769,96.4,92.3,99.7,99.0,74.9,96.2
2,Harvard University,United States,21887,9.6,0.25,1.0000000,95.2,94.8,99.0,99.3,49.5,80.5
3,University of Cambridge,United Kingdom,20185,11.3,0.39,0.8867925,94.8,90.9,99.5,97.0,54.2,95.8
3,Stanford University,United States,16164,7.1,0.24,0.8518519,94.8,94.2,96.7,99.8,65.0,79.8
5,Massachusetts Institute of Technology,United States,11415,8.2,0.33,0.6666667,94.2,90.7,93.6,99.8,90.9,89.3
6,California Institute of Technology,United States,2237,6.2,0.34,0.5873016,94.1,90.9,97.0,97.3,89.8,83.6


### part b

We will first fit the full linear model.

In [96]:
# fitting the linear model
university_lm <- lm(overall_score ~ . - university_rank - name_of_university - location, data = university_data_cleaned)
uni_tidy_lm <- tidy(university_lm)
uni_tidy_lm

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-1.838826e-01,2.461143e-01,-0.7471430,4.550967e-01
no_of_student,-7.673500e-07,1.527197e-06,-0.5024565,6.154217e-01
no_of_student_per_staff,-1.652109e-03,3.904165e-03,-0.4231657,6.722362e-01
international_student,-1.339878e-01,6.096873e-01,-0.2197648,8.260849e-01
female_male_ratio,2.273331e-02,6.805780e-02,0.3340294,7.384050e-01
teaching_score,2.960917e-01,7.275665e-03,40.6961716,3.549477e-243
research_score,3.041641e-01,6.680447e-03,45.5304958,3.617794e-283
citations_score,3.008538e-01,1.845641e-03,163.0077605,0.000000e+00
industry_income_score,2.254527e-02,3.752397e-03,6.0082307,2.360366e-09


In [97]:
# check their vifs
university_vif <- vif(university_lm)
university_vif

no_of_student     no_of_student_per_staff 
                   1.201010                    1.291369 
      international_student           female_male_ratio 
                   3.184319                    1.093170 
             teaching_score              research_score 
                   5.342592                    7.096826 
            citations_score       industry_income_score 
                   1.595709                    1.576786 
international_outlook_score 
                   3.838093

Research score and teaching score have concerningly high variance inflation factors, indicating we should remove these variables from the model. However, upon closer inspection, the two variables are highly correlated to each other, thus, removing the variable with the highest vif should suffice.

In [98]:
teaching_removed_lm <- lm(overall_score ~ . - university_rank - name_of_university - location - teaching_score, 
                          data = university_data_cleaned)
teaching_removed_lm
teaching_removed_vif <- vif(teaching_removed_lm)
teaching_removed_vif


Call:
lm(formula = overall_score ~ . - university_rank - name_of_university - 
    location - teaching_score, data = university_data_cleaned)

Coefficients:
                (Intercept)                no_of_student  
                  5.234e+00                    2.676e-06  
    no_of_student_per_staff        international_student  
                 -4.840e-02                    3.042e+00  
          female_male_ratio               research_score  
                  1.093e-01                    5.278e-01  
            citations_score        industry_income_score  
                  3.108e-01                    5.800e-03  
international_outlook_score  
                  3.814e-02  


no_of_student     no_of_student_per_staff 
                   1.197325                    1.179593 
      international_student           female_male_ratio 
                   3.132158                    1.092102 
             research_score             citations_score 
                   2.293924                    1.567646 
      industry_income_score international_outlook_score 
                   1.557826                    3.541962

However, using only the vif may not result in the best fitted model; thus, we will use regularization techniques to select the best variables to include in our final model. Since this data has a large number of covariates where multicollinearity could potentially be an issue and we want to find the best variables to predict future values of `overall_score`, we will run Lasso regularization to select our varaiables for the final model. 

First we will split our data into training(70%) and testing(30%) data.

In [99]:
data_split <- initial_split(university_data_cleaned, prop = 0.7)
uni_training <- training(data_split)
uni_testing <- testing(data_split)
head(uni_training)
head(uni_testing)

university_rank,name_of_university,location,no_of_student,no_of_student_per_staff,international_student,female_male_ratio,overall_score,teaching_score,research_score,citations_score,industry_income_score,international_outlook_score
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
183,Autonomous University of Barcelona,Spain,31933,12.7,0.16,1.5000000,55.60,35.1,38.8,91.1,43.0,67.1
156,TU Dresden,Germany,30382,30.8,0.16,0.8181818,57.40,48.8,50.7,69.1,92.4,60.5
601–800,Konkuk University,South Korea,13375,26.2,0.14,0.9607843,36.60,30.6,37.8,30.5,45.8,42.1
90,Sorbonne University,France,41443,13.9,0.21,1.4390244,64.50,58.7,58.3,76.6,40.0,72.3
501–600,University of Tsukuba,Japan,15725,12.7,0.16,0.6129032,40.65,43.9,37.8,38.5,43.7,43.0
1001–1200,University of Szeged,Hungary,20023,12.5,0.22,1.2222222,27.05,23.2,15.5,33.3,39.6,60.8


university_rank,name_of_university,location,no_of_student,no_of_student_per_staff,international_student,female_male_ratio,overall_score,teaching_score,research_score,citations_score,industry_income_score,international_outlook_score
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
6,California Institute of Technology,United States,2237,6.2,0.34,0.5873016,94.1,90.9,97.0,97.3,89.8,83.6
7,Princeton University,United States,8279,8.0,0.23,0.8518519,92.4,87.6,95.9,99.1,66.0,80.3
8,"University of California, Berkeley",United States,40921,18.4,0.24,1.0833333,92.1,86.4,95.8,99.0,76.8,78.4
21,"University of California, Los Angeles",United States,42434,9.7,0.16,1.2727273,85.8,80.4,88.9,95.4,58.8,65.0
26,University of Washington,United States,47727,10.8,0.18,1.2222222,82.1,71.6,82.8,98.9,53.9,63.0
36,"Nanyang Technological University, Singapore",Singapore,24651,15.1,0.25,0.9230769,77.0,60.9,77.9,87.2,84.5,94.5


Now we will run Lasso regularization on `uni_training` since the training data is used to train the model to fit the data and penalize large coefficients. By running lasso on the training data we will be able to find the value of the L1 penalty term ($\lambda$) that provides the lowest cross-validation MSE.

In [100]:
uni_lasso<-
    cv.glmnet(uni_training %>% select(-university_rank, -name_of_university, -location, -overall_score) %>% as.matrix(), 
              uni_training$overall_score, 
              alpha=1)

lasso_model


Call:  cv.glmnet(x = uni_testing %>% select(-university_rank, -name_of_university,      -location, -overall_score) %>% as.matrix(), y = uni_testing$overall_score,      alpha = 1) 

Measure: Mean-Squared Error 

    Lambda Index Measure      SE Nonzero
min 0.2205    44  0.1915 0.04553       5
1se 0.2420    43  0.2268 0.05453       5

Now that we ran the model, we will extract the coefficients of the model with the lowest cross-validation MSE and the names of the covariates.

In [101]:
lasso_coef <-
    coef(uni_lasso, s = uni_lasso$lambda.min)

lasso_selected_covariates <-
    as_tibble(
        as.matrix(lasso_coef),
        rownames='covariate') %>%
        filter(covariate != '(Intercept)' & abs(s1) !=0) %>% 
        pull(covariate)
lasso_coef
lasso_selected_covariates

10 x 1 sparse Matrix of class "dgCMatrix"
                                    s1
(Intercept)                 0.12432155
no_of_student               .         
no_of_student_per_staff     .         
international_student       .         
female_male_ratio           .         
teaching_score              0.28629476
research_score              0.31195223
citations_score             0.30045131
industry_income_score       0.01653456
international_outlook_score 0.07846905

[1] "teaching_score"              "research_score"             
[3] "citations_score"             "industry_income_score"      
[5] "international_outlook_score"

Lasso regression should remove variables with high multicollinearity and is able to fit models on datasets with high multicollinearity. To verify this, we will check the variance inflation factor (VIF) of the lasso model

In [102]:
lasso_vif <- vif(lm(overall_score ~ . , data = uni_training %>% 
        select(contains(lasso_selected_covariates), overall_score)))
lasso_vif

teaching_score              research_score 
                   5.111544                    7.150019 
            citations_score       industry_income_score 
                   1.548980                    1.589990 
international_outlook_score 
                   1.755476

Most of the variables selected in this model have low vif values indicating there is no issue of multicollinearity. However, `teaching_score` and `reasearch_score` both have high vif values which point to multicollinearity. Despite the high multicollinearity, the model could still be effective since lasso regression includeds variables that have high multicollinearity if they contribute significantly to the model. To verify the effectiveness of the model, we will fit an oridinary least squares model with the lasso selected variables and evaluate the $R^2$ value to determine the models effectiveness. To fit the OLS we will use the testing data which was kept separate from the training process as it provides an unbiased evaluation when comparing different models and a measure of how well the model does on unseen data. 

In [103]:
inference_model <- 
    lm(overall_score ~ .,
        data = uni_testing %>% 
                   select(contains(lasso_selected_covariates), overall_score))
summary(inference_model)


Call:
lm(formula = overall_score ~ ., data = uni_testing %>% select(contains(lasso_selected_covariates), 
    overall_score))

Residuals:
    Min      1Q  Median      3Q     Max 
-3.9253 -0.9410  0.0747  1.0298  9.8302 

Coefficients:
                            Estimate Std. Error t value Pr(>|t|)    
(Intercept)                 0.045040   0.362985   0.124 0.901308    
teaching_score              0.305862   0.012139  25.196  < 2e-16 ***
research_score              0.295621   0.011346  26.055  < 2e-16 ***
citations_score             0.298898   0.003349  89.238  < 2e-16 ***
industry_income_score       0.025062   0.006746   3.715 0.000229 ***
international_outlook_score 0.073828   0.004194  17.605  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.535 on 439 degrees of freedom
Multiple R-squared:  0.9899,	Adjusted R-squared:  0.9898 
F-statistic:  8588 on 5 and 439 DF,  p-value: < 2.2e-16


We will now compare it to the full OLS model.

In [105]:
summary(university_lm)


Call:
lm(formula = overall_score ~ . - university_rank - name_of_university - 
    location, data = university_data_cleaned)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.7973 -1.0143  0.0117  1.0134 10.1299 

Coefficients:
                              Estimate Std. Error t value Pr(>|t|)    
(Intercept)                 -1.839e-01  2.461e-01  -0.747    0.455    
no_of_student               -7.673e-07  1.527e-06  -0.502    0.615    
no_of_student_per_staff     -1.652e-03  3.904e-03  -0.423    0.672    
international_student       -1.340e-01  6.097e-01  -0.220    0.826    
female_male_ratio            2.273e-02  6.806e-02   0.334    0.738    
teaching_score               2.961e-01  7.276e-03  40.696  < 2e-16 ***
research_score               3.042e-01  6.680e-03  45.530  < 2e-16 ***
citations_score              3.009e-01  1.846e-03 163.008  < 2e-16 ***
industry_income_score        2.255e-02  3.752e-03   6.008 2.36e-09 ***
international_outlook_score  7.886e-02  3.603e-03  21.8

The adjusted $R^2$ in the full model is slightly higher than the model with the lasso selected variables which implies that the full model explains a slighlty higher proportion (0.9903) of the variance of `overall_score` than the model with lasso selected variables (0.9898). Since the $R^2$ between the full model and lasso model was relatively the same, it justifies the variables selected with the lasso model since it implies that the variables removed did not make a significant impact to the models ability to explain the variance in `overall_score`. This result is also consistent with the result of the $p$-values in the full model as the variables that were removed in the lasso regression are also shown as insignificant in the full model.

### Discussion

In this section, you’ll interpret the results you obtained in the previous section with respect to the main question/goal of your project.

Summarize what you found and the implications/impact of your findings.
If relevant, discuss whether your results were what you expected to find.
Discuss how your model could be improved;
Discuss future questions/research this study could lead to.

### References

At least two citations of literature relevant to the project. The citation format is your choice – just be consistent. Make sure to cite the source of your data as well.